In [ ]:
import cv2 as cv
import os 
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from dotenv import load_dotenv
import MySQLdb
import numpy as np
import base64
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

### Database

In [ ]:
# Load up the .env file
try:
    if (load_dotenv()):
        print("Loaded .env variables")
    else:
        print("Error in loading variables or file not found!")

except Exception:
    print("Error in loading .env file")
## Connect to database
host = os.environ.get("DB_HOST")
user = os.environ.get("DB_USER")
pw = os.environ.get("DB_PASS")
db_name = os.environ.get("DB_SCHEMA")
try:
    connection = MySQLdb.connect(host, user, pw, db_name)
    print("Successfully connected to ", db_name)
    connection.close()
except Exception:
    print("Error connecting!")
    
# Create an instance for the db and cursor
def getDBCursor():
    db = MySQLdb.connect(host, user, pw, db_name)
    cursor = db.cursor()
    return cursor

### Fetching all captures from database

In [ ]:
# Define a global variable to store all IMG bytearray and data
face_capture = []
face_img = []

# Face Entity and all Facts from the table 
face_id = []
tenant_id = []
tenant_name = []
face_status = []


def fetch_all_FaceCaptures():
    '''
        Begin making queries to the database and SELECT our FACE_IMG table
        where all facial captures are stored
    '''
    # Create a query to select all facial captures
    sql_fetch_faceCaptures = "SELECT * FROM FACE_IMG"
    cursor = getDBCursor()
    cursor.execute(sql_fetch_faceCaptures)
    # Fetch all data as rows
    rows = cursor.fetchall()

    # Iterate and assign data as to row
    for row in rows:
        face_id = row[0]
        tenant_id = row[1]
        tenant_name = row[2]
        face_status = row[3]
        face_capture = row[4]
        # Convert the face_capture column to a bytes-like object
        face_capture_bytes = bytearray(face_capture)
        # Use the cv.imdecode() function to decode the bytes-like object into an image
        face_img = cv.imdecode(np.frombuffer(face_capture_bytes, np.uint8), cv.IMREAD_COLOR)

        # Display some attribute data 
        print("==============================================================")
        print("Face ID: ", face_id)
        print("Tenant ID: ", tenant_id)
        print("Tenant Full Name: ", tenant_name)
        print("Status: ", face_status)
        print("Face IMG data: ", face_img)
        print(" ---> Type: ", type(face_img))
        print("==============================================================")

        # Render the image 
        try:
            # Convert image to PNG format and store it in a variable
            try: 
                # encoded_img = cv.imencode(".png", face_img)
                # print("Converting image to .png")
                # face_img_png = encoded_img.tobytes()
                # with open(f'out/{tenant_name}.png', 'wb') as f:
                #     # Decode the base64-encoded PNG data
                #     decoded_data = base64.b64decode(face_img_png)
                #     print(f"Saved PNG file: out/{tenant_name}.png")
                #     # Write the decoded data to the file stream
                #     f.write(decoded_data)
                if((os.path.exists('dataset/train/images'))):
                        cv.imwrite(f'dataset/train/images/{tenant_name}.png', face_img)
                        print("Images downloaded to", f'dataset/train/images/{tenant_name}.png')
                else: 
                    cv.imwrite(f'dataset/train/images/{tenant_name}.png', face_img)
                    os.mkdir('dataset/train/images')
                    print("Images downloaded to", f'dataset/train/images/{tenant_name}.png')
            except:
                print("An error has occurred converting the image")
        except: 
            print("There was an error to rendering the image")

# Call the function to test it
fetch_all_FaceCaptures()

### Previewing our image

In [ ]:
img = cv.imread("dataset/train/images/Cloyd Van Secuya.png")
# opencv BGR channel format and plt reads images as RGB channel format

In [ ]:
img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
plt.imshow(img) # RGB 

### Face Detection

Using MTCNN we can initiate to detect faces from our images

In [ ]:
from mtcnn.mtcnn import MTCNN

detector = MTCNN()
results = detector.detect_faces(img)

Viewing an array of results

In [ ]:
results

Getting the bounding areas

In [ ]:
x,y,w,h = results[0]['box']

Cropping the image based on bounding area

In [ ]:
img = cv.rectangle(img, (x,y), (x+w, y+h), (0,0,255), 30)
plt.imshow(img)

In [ ]:
my_face = img[y:y+h, x:x+w]
#Facenet takes as input 160x160 
my_face = cv.resize(my_face, (160,160))
plt.imshow(my_face)

### Previewing the cropped image

In [ ]:
my_face

### Creating a template for pre-processing 


Here we need to define a class to load, extract, and fetch our images

In [ ]:
class FACELOADING:
    ''' We need to read through the ../out directory '''
    def __init__(self, directory):
        print("Going into, ", directory)
        self.directory = directory
        self.target_size = (160,160)
        self.X = []
        self.Y = []
        self.detector = MTCNN()
    

    def extract_face(self, filename):
        print("Reading images ====> ", filename)
        img = cv.imread(filename)
        print("Converting to RGB Channels")
        img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
        x,y,w,h = self.detector.detect_faces(img)[0]['box']
        print("Bounding area: ")
        print("\t ===================")
        print("\t x: ", x)
        print("\t y: ", y)
        print("\t width: ", w)
        print("\t height: ", h)
        print("\t ===================")
        x,y = abs(x), abs(y)
        face = img[y:y+h, x:x+w]
        print("Resizing: ", self.target_size)
        face_arr = cv.resize(face, self.target_size)
        return face_arr
    

    def load_faces(self, path):
      FACES = []
      print("Loading faces...")
      if not os.path.isfile(path):
          return FACES
      try:
          single_face = self.extract_face(path)
          FACES.append(single_face)
      except Exception as e:
          print("An error has occurred in loading facial capture images")
          pass
      return FACES


    def load_classes(self):
        for sub_dir in os.listdir(self.directory):
            path = os.path.join(self.directory, sub_dir)
            print("Loading images from: ", path)
            FACES = self.load_faces(path)
            labels = [sub_dir for _ in range(len(FACES))]
            self.X.extend(FACES)
            self.Y.extend(labels)
        return np.asarray(self.X), np.asarray(self.Y)

    def plot_images(self):
        print("Plotting images....")
        plt.figure(figsize=(18,16))
        for num,image in enumerate(self.X):
            ncols = 3
            nrows = len(self.Y)//ncols + 1
            plt.subplot(nrows,ncols,num+1)
            plt.imshow(image)
            plt.axis('off')

### Loading Faces

In [ ]:
faceloading = FACELOADING("dataset/train/images")
X, Y = faceloading.load_classes()

In [ ]:
plt.figure(figsize=(16,12))
for num,image in enumerate(X):
    ncols = 3
    nrows = len(Y)//ncols + 1
    plt.subplot(nrows,ncols,num+1)
    plt.imshow(image)
    plt.axis('off')

### FaceNet Implementation 

In [ ]:
from keras_facenet import FaceNet
embedder = FaceNet()

def get_embedding(face_img):
    face_img = face_img.astype('float32') # 3D(160x160x3)
    face_img = np.expand_dims(face_img, axis=0) 
    # 4D (Nonex160x160x3)
    yhat= embedder.embeddings(face_img)
    return yhat[0] # 512D image (1x1x512)

### Embeddings

In [ ]:
EMBEDDED_X = []

for img in X:
    EMBEDDED_X.append(get_embedding(img))

EMBEDDED_X = np.asarray(EMBEDDED_X)

In [ ]:
EMBEDDED_X

In [ ]:
EMBEDDED_X[0]

In [ ]:
np.savez_compressed('classes/faces_embeddings_done_4classes.npz', EMBEDDED_X, Y)

### SVM Modelling

In [ ]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
encoder.fit(Y)
Y = encoder.transform(Y)

In [ ]:
plt.plot(EMBEDDED_X[0]) 
plt.ylabel(Y[0])

Splitting the training and testing datasets

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(EMBEDDED_X, Y, shuffle=True, random_state=17)

Data Modelling

In [ ]:
from sklearn.svm import SVC
model = SVC(kernel='linear', probability=True)
model.fit(X_train, Y_train)

In [ ]:
ypreds_train = model.predict(X_train)
ypreds_test = model.predict(X_test)

Metrics

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(Y_train, ypreds_train)

In [ ]:
accuracy_score(Y_test,ypreds_test)

In [ ]:
t_im = cv.imread("dataset/test/images/Cloyd Van Secuya.png")
t_im = cv.cvtColor(t_im, cv.COLOR_BGR2RGB)
x,y,w,h = detector.detect_faces(t_im)[0]['box']

In [ ]:
t_im = t_im[y:y+h, x:x+w]
t_im = cv.resize(t_im, (160,160))
test_im = get_embedding(t_im)

In [ ]:
test_im = [test_im]
ypreds = model.predict(test_im)

In [ ]:
ypreds

In [ ]:
print(encoder.inverse_transform(ypreds))

Saving the model

In [ ]:
import pickle
#save the model
with open('model/svm_model_160x160.pkl','wb') as f:
    pickle.dump(model,f)